In [2]:
import os
import yaml
import itertools
import datetime
import shutil
import hydra
from omegaconf import DictConfig


def load_test_config(n_lst, p_lst, m_lst, noise_levels, model, cv_num, trial_num):
    config_list = []

    # Generate all combinations of n, p, and m
    combinations = list(itertools.product(n_lst, p_lst, m_lst, model))

    for combination in combinations:
        n_val, p_val, m_val, model_val = combination
        config = {
            "n": n_val,
            "p": p_val,
            "m": m_val,
            "noise_levels": noise_levels,
            "model": model_val,
            "cv_num": cv_num,
            "trial_num": trial_num,

        }
        config_list.append(config)
    return config_list


def load_model_config(
    signal_bag_percent,
    atom_bag_percent,
    select_atom_percent,
    replace_flag,
    agg_func,
    ignore_warning,
    random_seed,
    Bag_lst,
    K_lst,
):
    config_list = []

    # Generate all combinations of replace_flag
    combinations = list(itertools.product(replace_flag))

    for combination in combinations:
        replace_flag_val = combination[0]
        signal_bag_percent_val = [
            val for val in signal_bag_percent if val <= 1 or replace_flag_val
        ]
        config = {
            "signal_bag_percent": signal_bag_percent_val,
            "atom_bag_percent": atom_bag_percent,
            "select_atom_percent": select_atom_percent,
            "replace_flag": replace_flag_val,
            "agg_func": agg_func,
            "ignore_warning": ignore_warning,
            "random_seed": random_seed,
            "Bag_lst": Bag_lst,
            "K_lst": K_lst,
        }
        config_list.append(config)

    return config_list

def create_yaml_and_shell(
    test_config, model_config, configs_path, hydra, core_num, walltime, mail_address
):
    os.makedirs(configs_path, exist_ok=True)
    shell_script_path = "./shell_scripts"
    os.makedirs(shell_script_path, exist_ok=True)
    merged_config = itertools.product(test_config, model_config)
    formatted_date = datetime.date.today().strftime("%m%d")
    shell_files = []
    for config in merged_config:
        temp_test_config, temp_model_config = config
        temp_config = {"TEST": temp_test_config, "MODEL": temp_model_config}
        if hydra:
            temp_config["hydra"] = {
                "hydra_logging": {"level":"CRITICAL"},
                "job_logging": {"level":"CRITICAL"},
                "run": {
                    "dir": "memory/" + formatted_date + "/",
                },
            }
        with_replace_flag = temp_model_config["replace_flag"]
        filename = f"{temp_test_config['model']}_{temp_test_config['n']}_{temp_test_config['p']}_{temp_test_config['m']}_{['nr','r'][with_replace_flag]}_{formatted_date}.yaml"
        temp_config["filename"] = filename
        # output_path = [no_replacement_path, with_replacement_path][
        #     temp_model_config["replace_flag"]
        # ]
        output_path = configs_path
        with open(os.path.join(output_path, filename), "w") as file:
            yaml.dump(temp_config, file)

        script_base = f"""
        #!/bin/sh
        #SBATCH --account=stats
        #SBATCH --job-name={temp_test_config['model']}_{temp_test_config['n']}_{temp_test_config['p']}_{temp_test_config['m']}_{formatted_date}
        #SBATCH -c {core_num}
        #SBATCH -t {walltime}
        #SBATCH -C mem192
        #SBATCH --mail-type=ALL 
        #SBATCH --mail-user={mail_address}

        module load anaconda
        #Command to execute Python program
        python BOMP_testing.py --config-name {output_path}/{filename}
        #End of script
        """
        shell_filename = f"{temp_test_config['model']}_{temp_test_config['n']}_{temp_test_config['p']}_{temp_test_config['m']}_{['nr','r'][temp_model_config['replace_flag']]}_{formatted_date}.sh"
        shell_files.append(shell_filename)

        with open(os.path.join(shell_script_path, shell_filename), "w") as file:
            file.write(script_base)

    print("Don't forget to upload the configs to the server!")

    print("DOn't forget to upload the shell scripts to the server!")

    print("Don't forget to submit the jobs to the server!")

    print(f"Make sure you have the right email address: {mail_address}")

    print("Just copy and paste the following commands to the terminal:")
    for shell_file in shell_files:
        print(f"sbatch {shell_file}")

In [4]:
test_config = load_test_config(
    n_lst=[600],
    p_lst=[1000],
    m_lst=[20],
    noise_levels=[0.05],
    model=["BOMP"],
    cv_num=5,
    trial_num=2,
)
model_config = load_model_config(
    signal_bag_percent=[0.1,0.3,0.5,0.7,0.9,1.1,1.3,1.5],
    atom_bag_percent=[0.1,0.3,0.5,0.7,0.9],
    select_atom_percent=0,
    replace_flag=[True, False],
    agg_func="weight",
    ignore_warning=True,
    random_seed=1,
    Bag_lst=[1, 50, 100, 200, 300],
    K_lst=list(range(10, 300, 10)),
)

# merge the two configs

configs_path = "./configs"
hydra = True
core_num = 16
walltime = "5-00:00"
mail_address = "sz3091@columbia.edu"

Remove_configs = True
if Remove_configs:
    shutil.rmtree("configs")


create_yaml_and_shell(
    test_config, model_config, configs_path, hydra, core_num, walltime, mail_address
)

Don't forget to upload the configs to the server!
DOn't forget to upload the shell scripts to the server!
Don't forget to submit the jobs to the server!
Make sure you have the right email address: sz3091@columbia.edu
Just copy and paste the following commands to the terminal:
sbatch BOMP_600_1000_20_r_0714.sh
sbatch BOMP_600_1000_20_nr_0714.sh


In [15]:
filename = "BOMP_600_1000_20_20230713.yaml"
filename.split(".")[0]

'BOMP_600_1000_20_20230713'